In [10]:
import torch
import numpy as np
import pickle
import pandas as pd

from HMM_Likelyhood import HMM_log_likelihood,Optimizationloop, InitializeParameters,Accumulated_HHM_LL
import LogLikelihoodTorch

import HMM_Likelyhood

/Users/frederikkeuldahl/Desktop/Fagprojekt/GitHub/HMM


In [8]:

with open('GMMsubsetData.pickle','rb') as f:
    Xtrain = pickle.load(f) 
torch.set_default_dtype(torch.float64)

/Users/frederikkeuldahl/Desktop/Fagprojekt/GitHub/GeneralLinearModel


In [ ]:
#Fit HMM, K = 10, Subjects = 10
K=10

Pi_MM,Kappa_MM,mu_MM = LogLikelihoodTorch.InitializeFF(Xtrain,90,K)

Parameters_MM = [
{'params':Pi_MM},
{'params':Kappa_MM},
{'params':mu_MM},
]

AdamMM = torch.optim.Adam(Parameters_MM,lr=0.8)
AdamMM.zero_grad()
LogLikelihoodTorch.Optimizationloop(Xtrain,[Pi_MM,Kappa_MM,mu_MM],lose=LogLikelihoodTorch.log_likelihood,Optimizer=AdamMM,n_iters=2000,K=K)

Kappa_HMM,mu_HMM,Tk_HMM,Pinit_HMM = HMM_Likelyhood.InitializeParametersFF(Xtrain,10,90,K)
Parameters_HMM= [
{'params':Pinit_HMM},
{'params':Kappa_HMM},
{'params':mu_HMM},
{'params':Tk_HMM},
]

AdamHMM=torch.optim.Adam(Parameters_HMM,lr=1.2)
AdamHMM.zero_grad()
HMM_Likelyhood.Optimizationloop(Xtrain,[Pinit_HMM,Kappa_HMM,mu_HMM,Tk_HMM],lose=HMM_Likelyhood.Accumulated_HHM_LLHalf,Optimizer=AdamHMM,n=10,n_iters=2000,K=K)

In [ ]:
B_MM = HMM_Likelyhood.Softmax(LogLikelihoodTorch.log_pdf(Xtrain,mu_MM,Kappa_MM,90).T)
B_MM = B_MM/B_MM.sum(0)
B_HMM = HMM_Likelyhood.log_pdf(Xtrain,mu_HMM,Kappa_HMM,90).T
B_HMM = B_HMM/B_HMM.sum(0)

In [ ]:
B = np.zeros((10,330*10))

for j in range(10):
    Prop_HMM_sub1 = torch.zeros((10,330))
    Prop_HMM_sub1[:,0] = HMM_Likelyhood.Softmax((HMM_Likelyhood.Softmax(Pinit_HMM[:,j])*B_HMM[:,j*330]))

    T1 = HMM_Likelyhood.Softmax(Tk_HMM[j])
    for i in range(1,330):
        Prop_HMM_sub1[:,i]= torch.nn.Softmax(0)((T1@Prop_HMM_sub1[:,i-1])*B_HMM[:,i])

    B[:,j*330:(j+1)*330] = Prop_HMM_sub1.detach().numpy()

In [ ]:
with open('HMM_B_K=10.pickle','wb') as f:
    pickle.dump([mu_HMM,Kappa_HMM,B], f)

In [ ]:
X = B.detach().numpy().T
print(X.shape)

In [ ]:
pd.DataFrame(X).to_csv('XHMM_B_K=10.csv')

In [ ]:
#Fit Pytorch fitted Misture Model, K = 10, Subjects = 10

In [13]:
# Setting Up Parameters For Fitting of K Multivariate Watson Distributions

K = 10
p = 90

pi,kappa,mu = LogLikelihoodTorch.InitializeFF(Xtrain,p=p,K=K)

In [17]:

learning_rate = 1
n_iters = 2000

#torch.autograd.set_detect_anomaly(False)
Parameters = [
    {'params':pi},
    {'params':kappa},
    {'params':mu}
]

Adam = torch.optim.Adam(Parameters,lr=learning_rate)

In [ ]:
#Optimizing Distribution:
Pi_fit,Kappa_fit,mu_fit = LogLikelihoodTorch.Optimizationloop(Xtrain,[pi,kappa,mu],lose=LogLikelihoodTorch.log_likelihood,Optimizer=Adam,n_iters=n_iters,K=K)

In [ ]:
# Contraining Parametes and Saving Fitted Model
Softmax = torch.nn.Softmax(0)
Softplus = torch.nn.Softplus()

pi_est = Softmax(Pi_fit)
kappa_est = Softplus(Kappa_fit)
mu_est = mu /torch.sqrt((mu_fit * mu_fit).sum(axis=0))

In [ ]:
import LogLikelihoodTorch

B = LogLikelihoodTorch.log_pdf(Xtrain,mu_est,kappa_est,p).T

B = B/B.sum(0)

In [ ]:
with open('Torch_B_K=10.pickle','wb') as f:
    pickle.dump([pi_est,kappa_est,mu_est,B], f)

In [ ]:
X = B.detach().numpy().T
print(X.shape)

In [ ]:
pd.DataFrame(X).to_csv('XTorch_B_K=10.csv')